In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)

from datetime import datetime, date

import warnings
warnings.filterwarnings('ignore')

In [ ]:
playstore = pd.read_csv('../input/google-play-store-apps/googleplaystore.csv')
playstore

In [ ]:
playstore_user = pd.read_csv('../input/google-play-store-apps/googleplaystore_user_reviews.csv')
playstore_user

### Data Cleaning

In [ ]:
playstore.info()

In [ ]:
sns.heatmap(playstore.isnull());

In [ ]:
playstore.isnull().sum()

In [ ]:
playstore.dropna(axis = 0, inplace = True)

In [ ]:
playstore.isnull().sum()

In [ ]:
playstore.info()

#### Category

In [ ]:
playstore['Category'].describe()

In [ ]:
print( len(playstore['Category'].unique()) , 'categories')

print("\n", playstore['Category'].unique())

#### Reviews

In [ ]:
playstore['Reviews'].describe()

In [ ]:
playstore['Reviews']

In [ ]:
# Convert to int
playstore['Reviews'] = playstore['Reviews'].apply(lambda x: int(x))

In [ ]:
playstore['Reviews'].describe()

#### Size

In [ ]:
playstore['Size'].describe()

In [ ]:
playstore['Size']

In [ ]:
print( len(playstore['Size'].unique()) , "categories")

print("\n", playstore['Size'].unique())

In [ ]:
playstore.Size.value_counts().head()

In [ ]:
len(playstore[playstore.Size == 'Varies with device'])

In [ ]:
# Replacing 'Varies with device' by nan
playstore['Size'].replace('Varies with device', np.nan, inplace = True)

In [ ]:
# Replacing 'k' and 'M'
playstore['Size'].replace('k', '', regex = True, inplace = True)
playstore['Size'].replace('M', '', regex = True, inplace = True)

In [ ]:
print( len(playstore['Size'].unique()) , "categories")

print("\n", playstore['Size'].unique())

In [ ]:
playstore.Size = pd.to_numeric(playstore.Size)

In [ ]:
# fill "Varies with device" with mean of size in each category
playstore['Size'].fillna(playstore.groupby('Category')['Size'].transform('mean'),inplace = True)

In [ ]:
playstore.Size.describe()

#### Installs

In [ ]:
print( len(playstore['Installs'].unique()) , "categories")

print("\n", playstore['Installs'].unique())

In [ ]:
playstore.Installs.value_counts()

In [ ]:
playstore.Installs = playstore.Installs.apply(lambda x: x.replace(',',''))
playstore.Installs = playstore.Installs.apply(lambda x: x.replace('+',''))
playstore.Installs = playstore.Installs.apply(lambda x: int(x))

In [ ]:
playstore.Installs.unique()

#### Type

In [ ]:
playstore.Type.value_counts()

#### Price

In [ ]:
playstore.Price.value_counts()

In [ ]:
print( len(playstore['Price'].unique()) , "categories")

print("\n", playstore['Price'].unique())

In [ ]:
playstore.Price = playstore.Price.apply(lambda x: x.replace('$',''))
playstore['Price'] = playstore['Price'].apply(lambda x: float(x))

In [ ]:
playstore.Price.describe()

In [ ]:
playstore.Price.unique()

#### Content Rating

In [ ]:
playstore['Content Rating'].unique()

In [ ]:
playstore['Content Rating'].value_counts()

#### Genres

In [ ]:
print( len(playstore['Genres'].unique()) , "categories")

print("\n", playstore['Genres'].unique())

In [ ]:
playstore.Genres.value_counts()

In [ ]:
# The data is divided between genre and subgenre, so I will only group by genre
playstore['Genres'] = playstore['Genres'].str.split(';').str[0]
playstore['Genres'].replace('Music & Audio', 'Music',inplace = True)

In [ ]:
print( len(playstore['Genres'].unique()) , "categories")

print("\n", playstore['Genres'].unique())

#### Last Updated

In [ ]:
playstore['Last Updated']

In [ ]:
# New column containing the converted dates
playstore['new'] = pd.to_datetime(playstore['Last Updated'])
playstore['new'].describe()

In [ ]:
playstore['lastupdate'] = (playstore['new'] -  playstore['new'].max()).dt.days
playstore['lastupdate'].head()

### EDA

In [ ]:
playstore.head()

In [ ]:
data = playstore
data.drop(['Current Ver', 'Android Ver', 'Last Updated', 'new', 'lastupdate'], axis = 1, inplace = True)
data.head()

#### Distributive relations

In [ ]:
fig = px.scatter_matrix(data, dimensions = ['Rating', 'Reviews', 'Size', 'Price', 'Installs'], color = 'Type')
fig.show()

##### **Free apps have the best ratings and the most installs. While the paid apps, despite the lowest installations, have the highest scores. There seems to be little correlation between the data, perhaps due to lack of user feedback.**

#### Numeric relevance

In [ ]:
fig = px.treemap(data, path = ['Rating', 'Type'])
fig.show()

In [ ]:
fig = px.treemap(data, path = ['Genres', 'Type'])
fig.show()

##### **The largest numbers of free apps are in the 'Tools', 'Entertainment' and 'Education' categories.**

In [ ]:
fig = px.treemap(data, path = ['Category'])
fig.show()

##### **Family, Games and Tools has the highest prevalence in the market.**

#### Average rating

In [ ]:
# Average rating
data.Rating.mean()

In [ ]:
fig = px.histogram(data, x = 'Rating')
fig.show()

#### Distribution

In [ ]:
hist = data.drop(['App'], axis = 1)
hist.hist(figsize = (20, 20), color = 'r');

In [ ]:
fig = px.histogram(data, x = 'Category', color = 'Category')
fig.show()

##### **The distribution of categories is significantly different**

#### Best performing categories

In [ ]:
fig = px.violin(data, x = 'Category', y = 'Rating', color = 'Category')
fig.show()

##### **All apps have a high average rating, with 'books and references' and 'events' having the highest average ratings!**

#### Haw the app sizes impact the app rating?

In [ ]:
fig = px.scatter(data, x = 'Size', y = 'Rating', marginal_x = 'histogram', marginal_y = 'histogram')
fig.show()

##### **Most highly rated apps are between 2 and 100 in size**

In [ ]:
fig = px.scatter(data, x = 'Price', y = 'Rating', marginal_x = 'histogram', marginal_y = 'histogram')
fig.show()

##### **Top rated apps are between zero and thirty dollars.**

#### Pricing trend by categorie

In [ ]:
fig = px.scatter(data, x = 'Price', y = 'Category', color = 'Category')
fig.show()

##### **I will check what these outliers are**

In [ ]:
print(data[data.Price > 200])

##### **The different versions of the 'I'm Rich' app appear to have considerable amounts of reviews, so I won't delete them.**

In [ ]:
# Filter "I'm Rich"
filt = data
filt = filt[filt['Price'] < 200]
fig = px.scatter(filt, x = 'Price', y = 'Category', color = 'Category')
fig.show()

##### **Family, lifestyle, sports and medical are the most expensive apps, with medical costing as much as 80 dollars! All others are under 30 dollars.**

#### Distribution of paid and free apps across categories

In [ ]:
fig = px.sunburst(data, path = ['Category', 'Type'], width = 800, height = 600)
fig.show()

#### Number of downloads of paid apps VS free apps

In [ ]:
filt = data.copy()
filt

In [ ]:
filt['Installs'] = np.log(filt['Installs'])
filt

In [ ]:
fig = px.box(filt, x = 'Type', y = 'Installs', color = 'Type')
fig.show()

##### **Paid apps have a relatively lower number of downloads than free apps. That is, obviously for monetary reasons, people prefer free apps.**

#### Paid and free app size variation

In [ ]:
fig = px.scatter(data, x = 'Rating', y = 'Size', color = 'Type', template = 'plotly_dark')
fig.show()

##### **Mostly, paid apps with higher rates have smaller sizes, as they are developed for specific functions, unlike a game, for example.**

#### Correlations

In [ ]:
filt = data.drop(['App', 'Category', 'Type', 'Content Rating', 'Genres'], axis = 1)

In [ ]:
corr = filt.corr()

In [ ]:
plt.figure(figsize = (10, 10))
sns.heatmap(corr, annot = True)

##### **There is a moderate positive relationship between the number of downloads and the number of reviews, that is, people tend to download a particular app according to the number of reviews, as this demonstrates the quality of the app. There is a positive but weak correlation between rating, size and downloads, that is, people do not care much about these aspects when downloading an app. The correlation between price and downloads is negative, that is, they are opposites. This means that the price does not influence the number of downloads, especially considering that most apps are free (to download).**

In [ ]:
filt = np.log(filt)
fig = px.scatter(filt, x = 'Reviews', y = 'Installs', trendline = 'ols', template = 'plotly_dark')
fig.show()

#### Linear Regression

In [ ]:
x = filt.iloc[:, 1].values

In [ ]:
y = filt.iloc[:, 3].values

In [ ]:
np.corrcoef(x, y)

In [ ]:
x = x.reshape(-1, 1)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x, y)

In [ ]:
predict = regressor.predict(x)
predict

In [ ]:
fig = px.scatter(x = x.ravel(), y = y)
fig.add_scatter(x = x.ravel(), y = predict, name = 'Regression')
fig.show()

In [ ]:
regressor.score(x, y)

In [ ]:
from yellowbrick.regressor import ResidualsPlot
visu = ResidualsPlot(regressor)
visu.fit(x, y)
visu.poof()

##### **Confirmed! The number of downloads can be explained and predicted by the number of reviews.**

### Sentiment Analysis

In [ ]:
playstore_user

In [ ]:
playstore_user.describe()

In [ ]:
playstore_user.info()

In [ ]:
playstore_user['Translated_Review']

In [ ]:
sns.heatmap(playstore_user.isnull())

In [ ]:
playstore_user.isnull().sum()

In [ ]:
playstore_user.dropna(axis = 0, inplace = True)

In [ ]:
playstore_user.isnull().sum()

#### Sentiment reviews

In [ ]:
fig = px.histogram(playstore_user, x = 'Sentiment', color = 'Sentiment', histnorm = 'density', template = 'plotly_dark')
fig.show()

#### Sentiment polarity distribution

In [ ]:
fig = px.box(playstore_user, x = 'Sentiment', y = 'Sentiment_Polarity', color = 'Sentiment', template = 'plotly_dark')
fig.show()

### Wordcloud

In [ ]:
positive = playstore_user[playstore_user['Sentiment'] == 'Positive']
positive

In [ ]:
negative = playstore_user[playstore_user['Sentiment'] == 'Negative']
negative

In [ ]:
sentences = playstore_user['Translated_Review'].tolist()
len(sentences)

In [ ]:
sentences_one_string = ' '.join(sentences)
sentences_one_string

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(sentences_one_string));

In [ ]:
negative_list = negative['Translated_Review'].tolist()
negative_sentences = ' '.join(negative_list)

In [ ]:
positive_list = positive['Translated_Review'].tolist()
positive_sentences = ' '.join(positive_list)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(positive_sentences));

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(negative_sentences));

##### **We can see that most feelings refer to the 'Games' category, as it appears in a larger size. Positive words like great, good, love appear. As for the negative words, we have propaganda, hate, bad, malware.**

# Conclusion

##### **The average rating of apps on the Google Play Store is 4.19, where the amount of downloads is related to the amount of user reviews, mostly for free apps. Users prefer smaller sized apps whose main categories are Medical, Game and Family.**